## 4. AI Assisted Search - Final concept

Extract key information from NL query

In [1]:
from dotenv import load_dotenv
load_dotenv("../.env")

False

In [2]:
from openai import AzureOpenAI

import os

client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_KEY"),  
  api_version = "2025-01-01-preview",
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

In [3]:
from pydantic import BaseModel
from typing import List, Any
import json

class QueryResponse(BaseModel):
    date_start: str
    date_end: str
    amount: int
    category: str
    filter: str

In [4]:
def _chat_gpt(messages, model="gpt-4o", temp=0, topp=0.1):
    response = client.beta.chat.completions.parse(
        model=model,
        messages=messages,
        temperature=temp,
        max_tokens=2000,
        top_p=topp,
        response_format=QueryResponse
    )   
    
    return response.choices[0].message.parsed

In [5]:
import datetime

#today = datetime.datetime.today().strftime('%Y-%m-%dT%H:%M:%S')
today = "2025-02-10T18:00:00"

In [6]:
_user_message = f"""You are a helpful assistant.

Today is {today}.
You're task is to extract the key information from the user's search query.

## Instructions

### Date
1. Week start from sunday to saturday.
2. If date range is not specified, set as today.
3. Today day range of date_start and date_end is the same.

### Amount
1. If amount is not specified, set as 0.

### Category
1. Valid categories are:
    - travel: 출장, 해외출장
    - tuition: 교육, 영어교육, 외국어 교육
    - meal: 식대, 식비
    - office: 사무용품, 사무용품 구입
    - other: 기타
2. If category is not specified, set as empty string.

### Filter
1. Using the extracted information, generate a filter string in Azure AI Search filter format.
2. Schema of Azure AI Search fields are:
    - date: YYYY-MM-DDTHH:MM:SS
    - amount: integer
    - category: string
3. Add UTC +09:00 to date.

## Example
Today: 2022-02-01 10:00:00
User query: 지난 달 출장 100만원 이상 비용 품의서를 검색
Date start: 2022-01-01
Date end: 2022-01-31
Amount: 1000000
Category: travel
Filter: "date ge '2022-01-01T09:00:00' and date le '2022-02-01T08:59:59' and amount ge 1000000 and category eq 'travel'"

User query: """

In [7]:
def generate(user_query):
    messages = [{"role": "user", "content": _user_message + user_query}]
    structued_response = _chat_gpt(messages)

    return structued_response.filter

In [8]:
import json

with open("sample_query.json", "r") as f:
    samples = json.load(f)

In [9]:
user_query = samples[0]["query"]
print(user_query)

ai_search_filter = generate(user_query)
print(ai_search_filter)

이번 달 3000만원 이상의 품의서를 찾아줘
date ge '2025-02-01T09:00:00' and date le '2025-03-01T08:59:59' and amount ge 30000000


### Search integrated

In [10]:
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient   

In [11]:
import os
service_endpoint = os.getenv("AZSCH_ENDPOINT")  
credential = AzureKeyCredential(os.environ["AZSCH_KEY"])

#print(service_endpoint)

index_name = "gsexpense-index"
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)

In [12]:
def search_query(query, filter=None):

    results = search_client.search(  
        search_text=query,
        search_fields=["content", "title"],
        select=["id", "content", "title", "date", "category", "amount", "user"],
        filter=filter,
        query_language="ko-kr",
        top=10 # for limiting text search
    ) 
    
    print("Search Results:")
    for i, result in enumerate(results, 1): 
        print(f"{i}) {result['@search.score']:.3f}: {result['id']}, {result['title']}, {result['date']}, {result['amount']}, {result['category']}, {result['user']['dept']}")  


In [13]:
print(user_query)
search_query("*", ai_search_filter)

이번 달 3000만원 이상의 품의서를 찾아줘
Search Results:
1) 1.000: ID0005, 공사비 품위서, 2025-02-03T18:00:00Z, 30000000, construction, 개발1팀


In [14]:
for sample in samples:
    user_query = sample['query']
    ai_search_filter = generate(user_query)
    print(f"{user_query}: {ai_search_filter}")

    search_query("*", ai_search_filter)
    print()

이번 달 3000만원 이상의 품의서를 찾아줘: date ge '2025-02-01T09:00:00' and date le '2025-03-01T08:59:59' and amount ge 30000000
Search Results:
1) 1.000: ID0005, 공사비 품위서, 2025-02-03T18:00:00Z, 30000000, construction, 개발1팀

지난 달 100만원 이상의 영어 교육비 품의서를 찾아줘: date ge '2025-01-01T09:00:00' and date le '2025-02-01T08:59:59' and amount ge 1000000 and category eq 'tuition'
Search Results:
1) 1.000: ID0003, 교육비(회계팀) 품위서, 2025-01-11T18:00:00Z, 1100000, tuition, 회계팀

오늘 작성된 품의서를 찾아줘: date ge '2025-02-10T09:00:00' and date le '2025-02-11T08:59:59'
Search Results:
1) 1.000: ID0007, 해외 출장비 품위서, 2025-02-10T20:00:00Z, 5000000, travel, 개발2팀

지난 주에 작성된 500만원 이상의 품의서를 찾아줘: date ge '2025-02-02T09:00:00' and date le '2025-02-09T08:59:59' and amount ge 5000000
Search Results:
1) 1.000: ID0006, 공사비 품위서 2, 2025-02-04T18:00:00Z, 10000000, construction, 개발2팀
2) 1.000: ID0005, 공사비 품위서, 2025-02-03T18:00:00Z, 30000000, construction, 개발1팀

